# Libraries importation

In [1]:
# Import all libraries needed 

import matplotlib.pyplot as plt
import pandas as pd
import sys 
import scipy
import numpy as np

from scipy.signal import *
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA
from sklearn.cluster import OPTICS
from sklearn import metrics

ImportError: cannot import name 'OPTICS' from 'sklearn.cluster' (C:\Users\sylva\Anaconda3\lib\site-packages\sklearn\cluster\__init__.py)

In [ ]:
sys.path.insert(0, '../packaging_PIR')
from neural_data_treatment_pkg.PrintFunctions import *
from neural_data_treatment_pkg.AdaBandFlt import *

In [ ]:
# Enable inline plotting
%matplotlib inline
# Enable outline plotting
%matplotlib tk

# Data Loading (execute only the one corresponding to your device)

In [9]:
# file path of csv file

Location = r'/Users/sylva/Documents/SUPAERO/2A/PIR/Data/data_no_burst/539W6cbaseRaw.txt'

In [5]:
Location = r'/Users/louiseplacidet/Desktop/PIR/Data/new_spike_data/newdata/E18KABaseline_BcutV2groundAll.txt'

In [6]:
# complete this one with your own path

#Location = 

In [10]:
# create dataframe
df = pd.read_csv(Location, sep='\t',skiprows=[0,1,3] , index_col='%t           ')

C:\Users\sylva\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [11]:
df.columns

Index(['El 41       ', 'El 42       ', 'El 44       ', 'El 54       ',
       'El 55       ', 'El 56       ', 'El 58       ', 'El 86       ',
       'El 15       ', 'El 51       '],
      dtype='object')

# Defining the sampling frequency and the alignement method for the rest of the notebook

In [33]:
fs = 50000
align_method = 'indice_1er_depass'
y_lim_min = -25
y_lim_max = 25

# Cuting and filtering the signal

In [13]:
#####################################################################################################################
####  BANK OF PARTS OF DATA
size = 1000000
all_raw_data = df #Entire recording from all electrodes

#full_signal = df.iloc[:,1] #Entire recording from electrode 58
full_signal = df.loc[:size,'El 58       '] #Entire recording from electrode 58

electrode_ref = df.loc[:size,'El 15       ']

# Desired cutoff frequencies (in Hz).
lowcut = 100.0
highcut = 5000.0

#y = butter_bandpass_filter(df.iloc[:,1], lowcut, highcut, fs, order=6)
y = butter_bandpass_filter(df.iloc[:size,6], lowcut, highcut, fs, order=5)
y_ref = butter_bandpass_filter(df.iloc[:size,8],lowcut,highcut,fs,order=5)

filtereddf = pd.DataFrame(y)
filtereddf.index = df.index[:size]

filtereddf_ref = pd.DataFrame(y_ref)
filtereddf_ref.index = df.index[:size]


signal_filtered = filtereddf.iloc[:,0] #Entire recording filtered by bandpass, for one electrode
signal_filtered_ref = filtereddf_ref.iloc[:,0]


###########################
## Signal de 20s

xminspike = int(np.round(12548*(fs/1000)))
xmaxspike = int(np.round(13000*(fs/1000)))

burst_data = filtereddf.iloc[xminspike:xmaxspike,0]

C:\Users\sylva\Anaconda3\lib\site-packages\scipy\signal\_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]


## selecting the signal or the part of the signal

In [14]:
signal = signal_filtered

# Run the noise initialisation

In [15]:
noise_levels = init_noise_levels(signal_filtered, fs, 
                                  noise_window_size = 0.01,
                                  required_valid_windows = 20,
                                  old_noise_level_propagation = 0.8, 
                                  test_level = 5,
                                  estimator_type = "RMS",
                                  percentile_value = 25,
                                  plot_estimator_graph = True)

# Find the spikes

In [16]:
spike_info = find_spikes(signal, noise_levels, fs,
                           window_size = 0.002,
                           noise_window_size = 0.01,
                           threshold_factor = 3.5,
                           positive_threshold_factor = 0.33,
                           maxseparation = 0.001,
                           time_checkmaxlocal = 0.0002)

# Record the spikes

In [17]:
spike_data = record_spikes(signal, fs, spike_info,
                              align_method,
                              t_before = 0.001,
                              t_after = 0.002)

In [34]:
print_spikes(spike_data,
             t_before_alignement = 0,
             first_spike = 0,
             last_spike = -1,
             fs = fs,
             randomize = True,
             nb_spike = 20,
             y_lim_min = y_lim_min,
             y_lim_max = y_lim_max)

In [19]:
spike_data_oneline = record_spikes_oneline(signal, fs, spike_info,
                                              align_method,
                                              t_before = 0.001,
                                              t_after = 0.002)

In [20]:
print_spikes_oneline(signal, spike_data_oneline)

# Bilan PCA + OPTICS

## PCA and OPTICS on spikes

In [25]:
from sklearn.preprocessing import StandardScaler

def PCA_and_OPTICS_spikes(spike_data, spike_info, nb_PCA_components=3, min_samples=5, max_eps=10, xi=0.05,min_cluster_size=5):
    
    ## on rééquilibre les valeurs dans les différentes dimensions
    #pca_data = np.array(spike_data.iloc[:,1:].values).transpose()
    #pca_data = StandardScaler().fit_transform(pca_data) # normalizing the features
    
    ## PCA
    pca_data = np.array(spike_data.iloc[:,1:].values).transpose()
    pca = PCA(n_components=nb_PCA_components)
    pca.fit(pca_data)
    PCA_X = pca.transform(pca_data)
    
    ## OPTICS
    
    optics = OPTICS(min_samples=min_samples, max_eps=max_eps, xi=0.05,min_cluster_size=min_cluster_size).fit(PCA_X)

    #core_samples_mask = np.zeros_like(optics.labels_, dtype=bool)
    #core_samples_mask[optics.core_sample_indices_] = True
    labels = optics.labels_

    # Number of clusters in labels, ignoring noise if present.
    n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
    n_noise_ = list(labels).count(-1)

    print('Estimated number of clusters: %d' % n_clusters_)
    print('Estimated number of noise points: %d' % n_noise_)
    print('')
    print('Number of spikes detected by AdaBandFlt: %d' % len(spike_info))
    print('Spikes placed into the clusters: %d' % (len(spike_info)-n_noise_))
    print('Percentage placed: %d ' % ((len(spike_info)-n_noise_)/len(spike_info)*100))
    
    ## Ajout du label des clusters dans spike info
    spike_info['cluster_label'] = optics.labels_
    
    return PCA_X, optics, spike_info
    

In [26]:
PCA_X, optics, updated_spike_info = PCA_and_OPTICS_spikes(spike_data, spike_info,5,3,500,0.05,3)
labels = optics.labels_

Estimated number of clusters: 13
Estimated number of noise points: 333

Number of spikes detected by AdaBandFlt: 389
Spikes placed into the clusters: 56
Percentage placed: 14 


## Plotting the PCA

In [25]:
PCA_plot(PCA_X)
print_clusters_3d(labels, PCA_X)

# Print the spikes clusterized

In [35]:
print_spikes_clusterized(spike_data,
                             labels,
                             t_before_alignement = 0.001,
                             nb_spike = 20,
                             y_lim_min = y_lim_min,
                             y_lim_max = y_lim_max,
                             fs = fs)

In [27]:
spike_data_clusterized_oneline = record_spikes_clusterized_oneline(signal, 
                                                                      fs, 
                                                                      spike_info,
                                                                      align_method,
                                                                       labels,
                                                                      t_before = 0.001,
                                                                      t_after = 0.002)

The labels located in labels will be used


In [36]:
print_spikes_clusterized_oneline(signal, spike_data_clusterized_oneline,
                             y_lim_min = y_lim_min,
                             y_lim_max = y_lim_max,)